# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [1]:
// Equivalent of adding dependencies to maven or sbt files
// For example, to add "org.scalaj" %% "scalaj-http" % "2.4.2" 
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

In [2]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

In [3]:
import $ivy.`org.json4s::json4s-jackson:3.4.0`

import $ivy.$                                 

In [4]:
import org.json4s.JsonDSL._
import org.json4s._

import org.json4s.JsonDSL._

import org.json4s._

In [5]:
import org.json4s.jackson.JsonMethods._

import org.json4s.jackson.JsonMethods._

In [6]:
import scala.util.matching.Regex

import scala.util.matching.Regex

In [7]:
import scala.collection.mutable.ArrayBuffer

import scala.collection.mutable.ArrayBuffer

### 1.1 Obtener el texto del RSS Feed

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

In [8]:
class ParseRSS {
    def request(url: String): String={
        val response: HttpResponse[String] = Http(url).timeout(connTimeoutMs = 2000, readTimeoutMs = 5000).asString
        response.body
    }
    def parseURL(responsebody : String): Seq[String]={
        val xml = XML.loadString(responsebody)
        // Extract text from title and description
        (xml \\ "item").map { item =>
    ((item \ "title").text +" " + (item \ "description").text)}
        
    }
    
}

class ParseJSON extends ParseRSS{
    implicit val formats = DefaultFormats
    override def parseURL(responsebody: String): Seq[String]={ //Cuerpo de respuesta completo
        val pattern ="(https?|ftp|file)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]".r //Patron
             //parse Reddit feed in JSON
            val jsonString = pattern.replaceAllIn(responsebody,"").trim() //Cuerpo de la respuesta sin urls
            val result = (parse(jsonString) \ "data" \ "children" \ "data")
    .extract[List[Map[String, Any]]] //Parses the jsonString as a List of Maps //Cuerpo de la funcion sin urls parseado en Json dentro de una [List[Map[String,Any]]]
            val filteredmap = result.map((_.filter((t) => t._1 == "title" || t._1 == "selftext").mapValues(_.asInstanceOf[String]).values)) //Filters the map and returns only "titles" and "selftexts"
            filteredmap.flatten.toSeq  //List[Seq[String,Any]] -(1)-> List[Seq[String,Any]] -(2)-> List[Seq[String,String]] -(3)-> List[Seq[String]] -(4)-> Seq[String]
    }
}

defined class ParseRSS
defined class ParseJSON

In [9]:
val rssmodel = new ParseRSS
val jsonmodel = new ParseJSON

rssmodel: ParseRSS = ammonite.$sess.cmd7$Helper$ParseRSS@5dc597ac
jsonmodel: ParseJSON = ammonite.$sess.cmd7$Helper$ParseJSON@75edf494

In [10]:
val rssresponsebody = rssmodel.request("https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc")
val jsonresponsebody = jsonmodel.request("https://www.reddit.com/r/Android/hot/.json?count=10")

rssresponsebody: String = """<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:media="http://search.yahoo.com/mrss/" xmlns:sy="http://purl.org/rss/1.0/modules/syndication/"><channel><title>Chicago Tribune</title><link>https://www.chicagotribune.com</link><language>en-US</language><copyright>© 2021 Chicago Tribune</copyright><atom:link href="https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:%5Bnow-2d+TO+now%5D&amp;sort=display_date:desc" rel="self" type="application/rss+xml"/><description>Chicago Tribune News Feed</description><lastBuildDate>Tue, 01 Jun 2021 22:35:52 +0000</lastBuildDate><ttl>1</ttl><sy:updatePeriod>hourly</sy:updatePeriod><sy:updateFrequency>1</sy:updateFrequency><item><title>NCAA athletes in Illinois could sign endorsement deals under bill passed by state lawmakers</title><link>https://ww

In [11]:
val parsedrss = rssmodel.parseURL(rssresponsebody)
val parsedjson = jsonmodel.parseURL(jsonresponsebody)

parsedrss: Seq[String] = List(
  "NCAA athletes in Illinois could sign endorsement deals under bill passed by state lawmakers Illinois college athletes would be able to hire agents and sign endorsement deals starting this summer under legislation passed by state lawmakers.",
  "Jos\u00e9 Abreu is named the American League Player of the Week after the Chicago White Sox first baseman had 10 RBIs in 7 games Chicago White Sox first baseman Jos\u00e9 Abreu earned American League Player of the Week honors for the Week of May 24-30 on Tuesday. It\u2019s his sixth career weekly honor.",
  "Column: The future is now for Adbert Alzolay, the exuberant Chicago Cubs pitcher fast becoming a fan favorite The Chicago Cubs haven't had a starting pitching prospect break through since the rebuild began in 2012. Adbert Alzolay is ready to stop the streak.",
  "Japan\u2019s vaccine push ahead of the Olympics looks to be too late and could \u2018trigger a global spread of different variants of the virus\u20

In [18]:
class FeedService[T <: ParseRSS]{
    var urls: Seq[(String,T)] = Seq[(String,T)]()
    def subscribe(urlTemplate: String,params: Seq[String],parser: T){ //ParseJSON
        val pat = "%s".r
        val urlparam = params.map {s => pat replaceAllIn(urlTemplate,s)}.map{url => (url,parser)}
        urls = urls ++ urlparam
    }
    def getText(): Seq[String] = {
        val texts = urls.map { url =>
            val parser = url._2 //parser = ParseJSON
            parser.parseURL(url._1)
            
        }
        texts.map(_.mkString(","))
    }
}


defined class FeedService

In [21]:
val feed = new FeedService[ParseRSS]

feed: FeedService[ParseRSS] = ammonite.$sess.cmd17$Helper$FeedService@45b5974e

In [22]:
feed.subscribe("https://www.chicagotribune.com/arcio/rss/category/%s/?query=display_date:[now-2d+TO+now]&sort=display_date:desc",Seq("sports","business"),rssmodel)

In [23]:
feed.urls

res22: Seq[(String, ParseRSS)] = List(
  (
    "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc",
    ammonite.$sess.cmd13$Helper$ParseRSS@523dfd90
  ),
  (
    "https://www.chicagotribune.com/arcio/rss/category/business/?query=display_date:[now-2d+TO+now]&sort=display_date:desc",
    ammonite.$sess.cmd13$Helper$ParseRSS@523dfd90
  )
)

In [21]:
feed.getText

: 

## 2. Detectar las entidades nombradas

### 2.1 Crear el modelo

El **modelo** es sólo la función `getNEs`, que recibe una lista de textos.
Para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

Este código lista los signos de puntuación y algunas palabras comunes del inglés que se van a sacar del texto.

In [13]:
class NERmodel{
     val STOPWORDS = Seq (
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
    "their", "theirs", "themselves", "what", "which", "who", "whom",
    "this", "that", "these", "those", "am", "is", "are", "was", "were",
    "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or",
    "because", "as", "until", "while", "of", "at", "by", "for", "with",
    "about", "against", "between", "into", "through", "during", "before",
    "after", "above", "below", "to", "from", "up", "down", "in", "out",
    "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each",
    "few", "more", "most", "other", "some", "such", "no", "nor", "not",
    "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
    "will", "just", "don", "should", "now", "on",
    // Contractions without '
    "im", "ive", "id", "Youre", "youd", "youve",
    "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
    "theyre", "theyd", "theyve",
    "shouldnt", "couldnt", "musnt", "cant", "wont",
    // Common uppercase words
    "hi", "hello"
)
    val punctuationSymbols = ".,()!?;:'`´\n"
    val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")
   
    //Funciones que obtienen los valores
    def getNEsSingle(text: String): Seq[String] =
  text.replaceAll(punctuationRegex, "").split(" ")
    .filter { word:String => word.length > 1 &&
              Character.isUpperCase(word.charAt(0)) &&
              !STOPWORDS.contains(word.toLowerCase) }.toSeq

    def getNEs(textList: Seq[String]): Seq[Seq[String]] = textList.map(getNEsSingle)
    
    //Aplicar pipeline
    def ExtractNEs(rssText: Seq[String]): Seq[Seq[String]] ={
            getNEs(rssText)
    }
    //Contar y ordenar
    def SortNEs(result: Seq[Seq[String]]): List[(String, Int)]={
        val counts: Map[String, Int] = result.flatten
  .foldLeft(Map.empty[String, Int]) {
     (count, word) => count + (word -> (count.getOrElse(word, 0) + 1)) }
         counts.toList
  .sortBy(_._2)(Ordering[Int].reverse)
        
    } 
}

defined class NERmodel

In [14]:
val model = new NERmodel

model: NERmodel = ammonite.$sess.cmd12$Helper$NERmodel@529d6a2c

### 2.2 Aplicar el "Modelo" a los datos

In [15]:
val resultjson = model.ExtractNEs(parsedjson)
val resultrss = model.ExtractNEs(parsedrss)

resultjson: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Note",
    "Join",
    "Ctrl-F",
    "Note",
    "Join",
    "IRC",
    "Telegram",
    "Monday",
    "Please",
    "Feel",
    "Home",
    "Smartphones",
    "Top",
    "Phones"
  ),
  ArrayBuffer("Weekly", "Superthread", "Jun"),
  ArrayBuffer(),
  ArrayBuffer("HTC", "Pixel", "XL"),
  ArrayBuffer(),
  ArrayBuffer("It\u2019s", "Flagship", "Exynos", "AMD", "RDNA2", "GPU"),
  ArrayBuffer(),
  ArrayBuffer("Nearby", "Shares", "Comes", "ChromeOS", "Share", "Android"),
  ArrayBuffer(),
  ArrayBuffer(
    "Easily",
    "Turn",
    "Fire",
    "Tablet",
    "Real",
    "Android",
    "Tablet",
    "HD10",
    "HD7",
    "HD8",
    "ETA",
    "PRIME"
  ),
...
resultrss: Seq[Seq[String]] = List(
  ArrayBuffer("NCAA", "Illinois", "Illinois"),
  ArrayBuffer(
    "Jos\u00e9",
    "Abreu",
    "American",
    "League",
    "Player",
    "Week",
    "Chicago",
    "White",
    "Sox",
    "RBIs",
    "Chicago",
    "White",
    "Sox",
    "Jos\u

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [16]:
val sortedNEsjson = model.SortNEs(resultjson)
val sortedNEsrss = model.SortNEs(resultrss)

sortedNEsjson: List[(String, Int)] = List(
  ("Android", 10),
  ("Google", 9),
  ("Note", 5),
  ("Join", 5),
  ("Samsung", 3),
  ("Telegram", 3),
  ("IRC", 3),
  ("Please", 2),
  ("OTA-Package", 2),
  ("Xperia", 2),
  ("Tablet", 2),
  ("Ctrl-F", 2),
  ("ETA", 2),
  ("AMOLED", 2),
  ("Monday", 2),
  ("Yes", 2),
  ("May", 2),
  ("III", 2),
  ("Nexus", 2),
  ("Sunday", 2),
  ("Fire", 2),
  ("Wear", 2),
  ("PRIME", 2),
  ("OS", 2),
  ("New", 2),
  ("Sony", 2),
  ("Recovery", 2),
  ("TV", 2),
  ("Advertisement", 1),
  ("Dimensity", 1),
  ("Comes", 1),
  ("LG", 1),
  ("Facebook\u2019s", 1),
  ("Chinese", 1),
  ("ROM", 1),
  ("HD7", 1),
  ("SHIELD", 1),
  ("Disagree", 1),
...
sortedNEsrss: List[(String, Int)] = List(
  ("Chicago", 23),
  ("Sox", 12),
  ("Cubs", 12),
  ("White", 10),
  ("Monday", 5),
  ("Naomi", 4),
  ("Open", 4),
  ("French", 4),
  ("Padres", 3),
  ("Sky", 3),
  ("Irving", 3),
  ("May", 3),
  ("Vaughn", 3),
  ("Week", 3),
  ("McClure", 3),
  ("Sunday", 3),
  ("Williams", 3),
